In [7]:
import numpy as np
from tensorflow import keras
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


knn on original input

In [8]:
#ref : https://towardsdatascience.com/multiclass-classification-using-k-nearest-neighbours-ca5281a9ef76

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

knn = KNeighborsClassifier()

#convert to 2d
x_train = np.reshape(x_train,(x_train.shape[0],-1))
x_test = np.reshape(x_test,(x_test.shape[0],-1))

knn.fit(x_train,y_train)

ypred = knn.predict(x_test)

result2 = accuracy_score(y_test,ypred)

print(result2)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.3398
